In [2]:
! pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [4]:
import json
import glob

# 아까 파일 찾기에 성공했던 그 경로 그대로 사용
ROOT_PATH = "data/aihub/**/*.json"

file_list = glob.glob(ROOT_PATH, recursive=True)

if len(file_list) > 0:
    # 첫 번째 파일만 딱 꺼내서 엽니다
    target_file = file_list[0]
    print(f"📂 파일 경로: {target_file}")
    
    with open(target_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    print("\n--- [JSON 내부 구조 확인] ---")
    # 보기 좋게 들여쓰기해서 출력 (앞부분 1000자만)
    print(json.dumps(data, indent=2, ensure_ascii=False)[:1000]) 
    
    print("\n-----------------------------")
    print("위 출력 결과에서 '리뷰 내용'과 '감정 라벨'이 어떤 영어 단어(Key)로 되어있는지 찾아야 합니다.")
else:
    print("❌ 경로에 파일이 없습니다.")

📂 파일 경로: data/aihub\Sample\02.라벨링데이터\SNS\01. 패션\1-1.여성의류(196).json

--- [JSON 내부 구조 확인] ---
[
  {
    "Index": "1024338",
    "RawText": "안녕하세요 이웃님들 반갑습니다. 요즘 날씨가 많이 쌀쌀해졌죠? 요즘 계절에 입으면 딱 좋을 인생 경량 패딩 하나 소개해 드릴게요.  다 함께 go go go~~  제가 입어보고 좋아서 엄마께도 구매해드렸네요. 딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서 어디서 샀냐고 많이들 물어보는 옷입니다.  엄마도 제가 입은 것 보시더니 탐내셔서  주문해드렸어요. 영하로 내려가는 날씨에는 이것만 입기엔 얇지만 초겨울까지는 운동 갈때 안에 얇은 기능성 반팔 입고 요것만 입어도 꽤 따뜻해요. 색상도 디자인도 무난해서 사무실에 두고 입기도 좋고, 평소에 가볍게 나갈때 코디하기도 좋아요. 앞으로 코트를 입거나 할때 안에 내피로 활용하기도 딱일 듯요! 맘같아선 색깔별로 쟁이고 싶습니다.  20대와 50대 모두 아우르는 디자인이 무난하고 깔끔하면서 고급스러운 옷 추천합니다~~~  설명 끝! 좀 도움이 되셨나요? 그러면 좋아요 꾹 눌러 주시고 전 이만 총총총~~ 항상 여러 이웃님들께 감사드립니다. ",
    "Source": "SNS",
    "Domain": "패션",
    "MainCategory": "여성의류",
    "ProductName": "OO 경량 다운 자켓",
    "Syllable": "513",
    "Word": "121",
    "GeneralPolarity": "1",
    "Aspects": [
      {
        "Aspect": "디자인",
        "SentimentText": "딱 기본 스타일인데 또 입은 거 보면 깔끔하게 저렴해보이지 않는 디자인이라서",
        "SentimentWord": "11",
        "SentimentPol

In [5]:
import json
import glob
import pandas as pd
import random 
from tqdm import tqdm

# ==========================================
# 1. 설정 (이 부분은 그대로 두세요)
# ==========================================
ROOT_PATH = "data/aihub/**/*.json"
TARGET_COUNT = 15000 

# ✅ 찾은 키 이름으로 수정 완료!
KEY_TEXT = "RawText"           
KEY_LABEL = "GeneralPolarity"  
# ==========================================

print(f"🔍 '{ROOT_PATH}' 경로에서 파일 검색 중...")
all_files = glob.glob(ROOT_PATH, recursive=True)

if len(all_files) == 0:
    print("❌ 파일을 찾지 못했습니다.")
else:
    print(f"✅ 검색된 전체 파일 수: {len(all_files)}개")

    print("🎲 파일 목록을 무작위로 섞는 중...")
    random.seed(42) 
    random.shuffle(all_files)

    print(f"🚀 상위 {TARGET_COUNT}개 파일에서 데이터 추출 시작...")
    
    collected_data = []
    success_count = 0

    for file_path in tqdm(all_files):
        if success_count >= TARGET_COUNT:
            break

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 📌 중요: 데이터가 리스트 [ ... ] 형태인 경우 처리
            # (방금 보여주신 구조가 리스트였습니다)
            if isinstance(data, list):
                items = data
            else:
                items = [data] # 리스트가 아니면 리스트로 감싸기

            for item in items:
                text = item.get(KEY_TEXT)
                org_label = str(item.get(KEY_LABEL)) # "1", "-1" 등 문자열로 변환

                # 📌 라벨 변환 로직 (AI Hub 쇼핑 데이터 기준)
                # "1" -> 긍정(1)
                # "-1" -> 부정(0)
                # "0" -> 중립 (학습에서 제외)
                
                final_label = None
                if org_label == "1":    # 긍정
                    final_label = 1
                elif org_label == "-1": # 부정
                    final_label = 0
                
                # 텍스트가 있고, 긍정/부정(1/0)이 확실한 경우만 저장
                if text and final_label is not None:
                    collected_data.append({'text': text, 'label': final_label})
                    success_count += 1
                
        except Exception as e:
            continue 

    # 저장
    if collected_data:
        df = pd.DataFrame(collected_data)
        print(f"\n🎉 최종 수집 완료: {len(df)}개")
        print("\n--- [데이터 샘플] ---")
        print(df.sample(5))
        
        # 긍정/부정 비율 확인
        print("\n--- [라벨 분포] ---")
        print(df['label'].value_counts())
        
        df.to_csv("aihub_shuffled_15k.csv", index=False, encoding='utf-8-sig')
        print("💾 저장 완료!")
    else:
        print("\n⚠️ 여전히 데이터가 0개입니다. 키 이름이나 폴더 구조를 다시 확인해야 합니다.")

🔍 'data/aihub/**/*.json' 경로에서 파일 검색 중...
✅ 검색된 전체 파일 수: 2385개
🎲 파일 목록을 무작위로 섞는 중...
🚀 상위 15000개 파일에서 데이터 추출 시작...


  9%|▊         | 206/2385 [00:00<00:03, 597.38it/s]


🎉 최종 수집 완료: 15023개

--- [데이터 샘플] ---
                                                    text  label
7830   허브 향 쓰다가 처음 쓰는 향인데 좋아하는 향은 아니라 좀 실망입니다.  그냥저냥 ...      0
10870  디자인 심플하니 세련되고 괜찮네요  사이즈도 태블릿으로 휴대성을 최대한 발휘하며 사...      1
1027   이 제품은 ooo 시그니처 향이라 익숙해서 좋았습니다. 매장에 들어가면 맡을 수 있...      1
8725   동생이 좋다고 해서 주문해 봤는데 머리에 떡 안 지고 자연스럽게 고정됩니다. 소프트...      1
213    실리콘이라 큰 냄비에도 작은 냄비에도 잘 맞고 찜기에 음식물이 들러 붙지 않아서 편...      1

--- [라벨 분포] ---
label
1    12357
0     2666
Name: count, dtype: int64
💾 저장 완료!
